## Checking If An Array is a View or Copy

In all the examples above, I showed whether we were working with a view or a copy by making an edit to the array in question and then looking at whether the original data had changed. But surely there's a more direct way to check if an array is a view or a copy, right?

Well... kind of. There is a way to identify when an array references data that was not created at the moment that array was created. Thus, in a sense, you can identify views. And we detail how that works below. **But:** crucially, this method is only able to identify arrays that reference other data, **not** whether the array is **sharing data with another array.** In other words, while we know (from our discussion above) that the creation of a view is best thought of as creating a mutual relationship between two arrays, the tool numpy provides to interrogate an array's view-copy status does not respect the mutuality of this relationship. 

### .base

All numpy arrays have a property called `.base` which points back to the original data referenced by the array if the array is a view (it is referencing data that wasn't created when it was created). If an array is not a view of some other array, then `.base` returns `None`.

In [66]:
my_vector = np.array([1, 2, 3, 4])
my_vector.base is None

True

Given that, we can define a simple function that tells us if an array is a view or not:

In [67]:
def array_is_a_view(x):
    if x.base is None:
        print("Not a view!")
    else:
        print("This is a view.")

array_is_a_view(my_vector)

Not a view!


Now we can use this function to confirm that simple indexing gets views:

In [68]:
array_is_a_view(my_vector[1:2])

This is a view.


And fancy indexing does not:

In [69]:
array_is_a_view(my_vector[[2, 1]])

Not a view!


### Views and Copies in Practice

But while this trick is useful for confirming your logic of what does and does not return a view, its usefulness in practice is a little limited.

That's because the reason we care about views and copies is because **we are interested in knowing whether the changes that we make to an array may propagate to other arrays.** And that's not something this function can really tell us. 

The reason is twofold. First, when a view is created, from numpy's perspective the newly created array is a view on the old array, while the original array is *not* a view:

In [70]:
original_vector = np.array([1, 2, 3])
new_vector = original_vector[0:2]

array_is_a_view(original_vector)

Not a view!


In [71]:
array_is_a_view(new_vector)

This is a view.


But as we've discussed repeatedly, that doesn't mean that changes to `original_vector` won't impact other arrays, and that changes to other arrays (e.g., `new_vector`) might not impact `original_vector`. In other words, this trick tells us something about how numpy has implemented the view, but not about *the existence of relationships between arrays*.

The second reason is that you can have things that, from an implementation perspective, are views, but where there is *no* danger of changes to the array impacting anything else. This actually happens a lot when you re-assign a subset to the original variable:

In [72]:
my_vector = np.array([1, 2, 3, 4])
array_is_a_view(my_vector)

Not a view!


In [73]:
my_vector = my_vector[2:4]
array_is_a_view(my_vector)

This is a view.


Here we created an array with values 1, 2, 3, and 4, then subset the third and fourth entries. This subset is a view of the original data, and so `.base` is not `None`. But because we assigned the subset back to the vector `my_vector`, while that original array of 1, 2, 3, and 4 still exists in memory, nothing can access it except through the variable `my_vector`, and so you don't have to worry about any changes elsewhere in your code propagating to `my_vector` even if it technically is a view.

So in conclusion, yes, numpy has some tools to tell you about what is a view and what's a copy *from an implementation perspective*. But given that what matters from the perspective of a data scientist is whether changes to an array may propagate to other arrays, or changes to other arrays may propagate back, that tooling isn't particularly helpful.